In [2]:
import requests
import xml.etree.ElementTree as ET
import re

import os
import shutil


import glob

In [3]:
import nltk
from nltk.corpus import words
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jmanu\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

## TRANSFOR THE PDF - XML & TEXT

In [10]:
import subprocess

# Paths should be raw strings to handle backslashes correctly
jar = "./Cermine_Java/cermine-impl-1.13-jar-with-dependencies.jar"
src = "./Data"

# Construct the command
command = ['java', '-cp', jar, 'pl.edu.icm.cermine.ContentExtractor', '-path', src, '-outputs', 'jats,text']

# Run the command
print(subprocess.run(command))

CompletedProcess(args=['java', '-cp', './Cermine_Java/cermine-impl-1.13-jar-with-dependencies.jar', 'pl.edu.icm.cermine.ContentExtractor', '-path', './Data', '-outputs', 'jats,text'], returncode=0)


## EXTRACTION DEISGN

In [4]:
def remove_duplicate_lines(file_path):
    seen_lines = set()
    unique_lines = []

    with open(file_path, 'r') as file:
        for line in file:
            clean_line = line.strip()
            if clean_line not in seen_lines:
                seen_lines.add(clean_line)
                unique_lines.append(line.strip())  # Keep the original line, including whitespace

    return unique_lines

In [5]:
def clean_sentence(sentence):
    # Remove numbers
    sentence = re.sub(r'\d+', ' ', sentence)

    # Remove special characters (excluding basic punctuation)
    sentence = re.sub(r'[^a-zA-Z\s,.?!()]', '', sentence)

    return sentence

In [6]:
def get_root(file):
    tree = ET.parse(file)
    root = tree.getroot()
    return root

In [7]:
def add_author(article, name, email=None, affiliation=None):
    new_author = {
        "Name": name,
        "Email": email if email else "none",
        "Affiliation": affiliation if affiliation else []
    }
    article['Author'].append(new_author)

In [8]:
def is_mostly_english(text, english_words, threshold=0.6):
    """
    Checks if most of the words in the text are English.
    
    :param text: The text to be checked.
    :param threshold: The fraction of words that need to be English.
    :return: True if the condition is met, False otherwise.
    """
    english_words = set(words.words())
    long_words = 0

    try:
        words_in_text = text.split()
        english_count = sum(1 for word in words_in_text if word.lower() in english_words)
    except:
        return text

    try:
        words_in_text = text.split()

        # Check for words that are longer of 20 characters 
        for word in words_in_text:
            if len(word) > 20:
                long_words += 1
    except:
        return text

    # Thresholds for english content and long words
    if not (english_count / len(words_in_text) >= threshold) or (long_words > 2):
        return "none"
    else:
        return text 


In [9]:
def get_abstract(root):
    abstract = "none"
    abs_state = 0
    abs_sec = 0

    # This will look for the abstract on the XML sections (outside of the metadata)
    if abs_state == 0:
        for sec in root.findall('.//sec'):
            sec_id = sec.get('id')  # Get the 'id' attribute of the <sec> element
            
            for itr in sec:
                
                # Test look at iterations
                #print(itr.tag)
                
                # When to stop recording
                if (abs_state == 1) and (

                    # Word starts with keyword, something different
                    (itr.text.strip().startswith(("ARTICLE HISTORY", "Keywords:", "KEYWORDS"))) or 
                    
                    # Changes of section
                    (abs_sec != sec_id) or 
                    
                    # It becomes a tittle
                    (itr.tag == 'title')
                ):
                    break
                    
                # Continue recording (Extra if conditions just in case)
                elif abs_sec == sec_id and abs_state == 1 and itr.tag == 'p':
                    abstract += '\n' + itr.text

                # First condition to find word abstract and change state to start recording
                if itr.text.strip().startswith(("Abstract", "ABSTRACT", "TECHNICAL ABSTRACT", "A B S T R A C T", "Executive Summary", 'a b s t r a c t')) and abs_state == 0:
                    abstract = itr.text
                    abs_state = 1
                    abs_sec = sec_id


            #article['Abstract'] = abstract
            if abs_state == 1:
                break

    return abstract

In [10]:
def remove_noise(text):
    # Define regex patterns for email and website URL
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
    website_pattern = r'https?://[^\s<>"]+|www\.[^\s<>"]+'

    # Split the text into words
    words = text.split()

    # FIRST PART REMOVE SINGLE ITEMS
    i = 0

    for i in range(0, len(words//4)):
        if len(words[i]) < 2 and len(words[i+1]) < 2:
            pass
        else:
            break

    words = words[i:]

    # Check if any of the first 10 words is an email or website
    for i, word in enumerate(words[:10]):
        if re.fullmatch(email_pattern, word) or re.search(website_pattern, word):
            # Return the text from the email or website onwards
            return ' '.join(words[i+1:])

    # If no email or website is found in the first 10 words, return the original text
    return " ".join(words)

In [11]:
def get_abstract_element(root):
    # This will try to find abstract as a found element on the XML
    try:
        if type(root.find('.//abstract/p').text) != None and len(root.find('.//abstract/p').text) > 30:
            text = root.find('.//abstract/p').text
            abstract = text
            abstract = remove_noise(abstract)
            return abstract
    except:
        return 'none' 

In [33]:
file = "C:\RWKV\Outputs\Txt\2020 A holistic methodology for development of Real-Time Digital Twins1-s2.0-S2212827120303449-main.cermtxt"
get_abstract_text(file)

'Did not read'

In [12]:

def get_abstract_text(file):
    # file = file[7:-8] 
    # try:
    #     text = remove_duplicate_lines(f"C:/Text/{file}.cermtxt")
    # except:
    #     return "Did not read"
    try:
        text = remove_duplicate_lines(file)
    except:
        return "Did not read"

    abstract = "none"
    abs_state = 0
    num_paragraphs = 0

    # Regex pattern to match initials (like "S. " or "L. A. ")
    initials_pattern = re.compile(r'\b[A-Z]\. ?[A-Z]?\.?')

    for sentence in text:
      

        # sentence = clean_sentence(sentence) 
        stop_characters = ["©", "*"]

        # Stopping conditions for Introduction
        if sentence.strip().startswith(("Introduction", "1 Introduction", "1. Introduction", ' 1 1.1 INTRODUCTION',  'I N T R O D U C T I O N')) and abs_state == 1:
            abs_state = 0

        # Stop keyword found after 5 paragraphs
        elif sentence.strip().startswith(("ARTICLE HISTORY", "Keywords:", "KEYWORDS", "Key", 'K E Y W O R D S', 'KXeywords')) and num_paragraphs > 5:
            abs_state = 0

        # Stops if © found on sentence after 5 sentences
        elif "©"  in sentence and num_paragraphs > 5:
            abs_state = 0

        # Stops if © found on sentence after 5 sentences
        elif "*"  in sentence and num_paragraphs > 5:
            abs_state = 0

        # Stops if found a figure, means I went on the text     
        elif "Fig." in sentence and abs_state == 1:
            print("You went on the text!!!!!!!")
            abs_state = 0

        # Max Len of Paragraphs    
        elif num_paragraphs > 30:
            abs_state = 0
        
        # More than 2 words all capital letters
        elif re.match(r'^([A-Z]{2,}\s+){1,}[A-Z]{2,}', sentence.strip()) and num_paragraphs > 5:
            abs_state = 0

        
        exclude_chars = ["@", "#"]
        
        # Starting conditions 
        if abs_state == 1 and not any(
            
            # Ecluding @ from sentence
            char in sentence for char in exclude_chars) and (

                # Excluding sentence with more than 7 numbers
                len(re.findall(r'\d', sentence)) <= 4) and not (
                    
                    # Excluding Names from sentence L. follow by something
                    initials_pattern.search(sentence)) and not (
                        
                        # More than 2 numbers starting 
                        re.match(r'^\d{2,}', sentence.strip())):
            
            # Add a paragraph
            num_paragraphs += 1
            abstract += '\n' + sentence

        if sentence.strip().startswith(("Abstract", "ABSTRACT", "TECHNICAL ABSTRACT", "A B S T R A C T", "Executive Summary", 'a b s t r a c t')) and abs_state == 0:
            abs_state = 1
            abstract = sentence

        


    return abstract

In [13]:
def get_abstract_last(root, english_words):

        abstract = ""
        num_paragraphs = 0
        exclude_chars = ["@", "#", "©", "*"]
        # Regex pattern to match initials (like "S. " or "L. A. ")
        initials_pattern = re.compile(r'\b[A-Z]\. ?[A-Z]?\.?')

        for sections in root.findall(".//sec"):
                for sec in sections:
                        for sentence in sec.text.split("\n"):
                                        if num_paragraphs < 40 and not any(
                                                
                                                # Ecluding @ from sentence
                                                char in sentence for char in exclude_chars) and (
                                                        
                                                        # Excluding sentence with more than 7 numbers
                                                        len(re.findall(r'\d', sentence)) <= 4) and not (
                                                        
                                                        # Excluding Names from sentence L. follow by something
                                                        initials_pattern.search(sentence)) and not (
                                                                
                                                                # More than 2 numbers starting 
                                                                re.match(r'^\d{2,}', sentence.strip())) and not (

                                                                        # Exclude website links
                                                                        re.compile(r'https?://\S+|www\.\S+|\S+\.com').search(sentence)) and not (
                                                                        
                                                                        # Eclude tems and condition
                                                                        "Terms & Conditions" in sentence):
                                                
                                                # Add a paragraph
                                                num_paragraphs += 1
                                                abstract += '\n' + sentence

                
                # Test if the paragraphs collected are good 
                try:
                        abstract = is_mostly_english(abstract, english_words)
                        if len(abstract.split()) > 25:
                                break
                                
                        else:
                                abstract = ""
                except:
                        abstract = ""
                
                        
                
        return abstract

In [14]:
def format_paragraph(paragraph):
    # Remove existing line breaks
    paragraph = paragraph.replace('\n', ' ')

    # Add a line break after each period that ends a sentence
    paragraph = re.sub(r'(\.\s)(?=[A-Za-z])', r'.\n', paragraph)

    return paragraph

In [15]:
# Extract all deatils to a new dictionary
def get_details(root, file, english_words):
    # Template structure
    article = {
        "Tittle" : "none",
        "Year": "none",
        "ID": "none",
        "Author": [],
        "Affiliations": {},
        "Abstract": ""
    }

    # State variables
    affilitations = {}
    aff_stat = 0
    abstract = ''
    
    # Extract article title
    try:
        article_title = root.find('.//article-title').text
        article['Tittle'] = article_title
        #article['Tittle'] = clean_sentece(article_title)
    except:
        pass

    # Extract Year
    try:
        year = root.find('.//pub-date/year').text
        article['Year'] = year
    except:
        pass

    # Extract DOI
    try:
        art_id = root.find('.//article-id').text
        article['ID'] = art_id
    except:
        pass

    # Find all Affilitaions
    try:
        for affiliation in root.findall('.//aff'):
            aff_stat = 1
            id = affiliation.get('id')
            institution = affiliation.find('institution').text
            affilitations[id] = institution

        article["Affiliations"] = affilitations
    except:
        pass

    # Find Name, Email, add Affiliations
    for author in root.findall('.//contrib[@contrib-type="author"]'):
        name = ""
        email = ""
        affiliation = []
        try:
            name = author.find('string-name').text
        except:
            pass

        try:
            email = author.find('email').text
        except:
            pass
        try:        
            if aff_stat == 1:
                for aff in author.findall('.//xref'):
                    id = aff.get('rid')
                    affiliation.append(affilitations[id])
        except:
            pass
    
        add_author(article, name, email, affiliation)
    
    # ABSTRACT EXTRACTIONS  
    # Method 1 get_abstract
    abstract = get_abstract(root)

    abstract = is_mostly_english(abstract, english_words)

    # Method 2 get_abstract_element
    try:
        if len(abstract.split()) < 50 or abstract == 'none':
            abstract = get_abstract_element(root)
    except:
        abstract = get_abstract_element(root)
    
    abstract = is_mostly_english(abstract, english_words)

    # Method 3 get_abstract_text
    try:
        if len(abstract.split()) < 50 or abstract == 'none':
            abstract = get_abstract_text(file)
    except:
        abstract = get_abstract_text(file)
    
    abstract = is_mostly_english(abstract, english_words)

    # Method 4 get_abstract_last
    try:
        if len(abstract.split()) < 50 or abstract == 'none':
            abstract = get_abstract_last(root, english_words)
    except:
        abstract = get_abstract_last(root, english_words)
    
    abstract = format_paragraph(abstract)
    
    article["Abstract"] = abstract


    return article

In [16]:
def load(file, text):
    english_words = set(words.words())
    root = get_root(file)
    article = get_details(root, text, english_words)

    return article

## Execute Processing

In [34]:
target_dir_xml = "./Outputs/Xml"
target_dir_txt = "./Outputs/Txt"
articles = []
not_work = []
for filename in os.listdir(target_dir_xml):
    xml_file = os.path.join(target_dir_xml, filename)
    txt_file = os.path.join(target_dir_txt, filename.replace(".cermxml", ".cermtxt"))
    filename_pdf = filename.replace(".cermxml", ".pdf")

    print("Processing file:", filename_pdf)
    try: 
        article = load(xml_file, txt_file)
        articles.append(article)
    except Exception as e:
        print(f"Error with file {filename_pdf}: {e}")
        not_work.append(filename_pdf)

# Print summary
print(f"Number of processed articles: {len(articles)}")
print(f"Number of files that failed to process: {len(not_work)}")
print(f"Files not processed:")
for files in not_work:
    print(files)

Processing file: 2020 A holistic methodology for development of Real-Time Digital Twins1-s2.0-S2212827120303449-main.pdf
Number of processed articles: 1
Number of files that failed to process: 0
Files not processed:


In [35]:
articles

[{'Tittle': 'A holistic me2th8tohdCoIRl oPgDyesifgonrCdonefvereelnocep, mMaeyn2t0o18f, RNaenatels-,TFriamncee Digital Twins A holistic methodology for development of Real-Time Digital Twins A new',
  'Year': '2019',
  'ID': 'none',
  'Author': [{'Name': 'Paul Stief',
    'Email': 'paul.stief@ensam.eu',
    'Affiliation': ["École Nationale Supérieure d'Arts et Métiers, Arts et Métiers ParisTech"]},
   {'Name': 'Jean-Yves Dantan',
    'Email': 'none',
    'Affiliation': ["École Nationale Supérieure d'Arts et Métiers, Arts et Métiers ParisTech"]},
   {'Name': 'Alain Etienne',
    'Email': 'none',
    'Affiliation': ["École Nationale Supérieure d'Arts et Métiers, Arts et Métiers ParisTech"]},
   {'Name': 'Ali Siadat',
    'Email': 'none',
    'Affiliation': ["École Nationale Supérieure d'Arts et Métiers, Arts et Métiers ParisTech"]}],
  'Affiliations': {'aff0': "École Nationale Supérieure d'Arts et Métiers, Arts et Métiers ParisTech",
   'aff1': 'FARO Europe GmbH & Co. 

## EXPORT to JSON

In [80]:
import json

# Convert the list of dictionaries to a JSON string
json_string = json.dumps(articles, indent=4)

# Print the JSON string
print(json_string)

# Optionally, save the JSON string to a file
with open("./Outputs/Extract/articles.json", "w") as file:
    file.write(json_string)

[
    {
        "Tittle": "A Review and Qualitative Meta-Analysis of Digital Human Modeling and Cyber-Physical-Systems in Ergonomics 4.0",
        "Year": "2021",
        "ID": "10.1080/24725838.2021.1966130",
        "Author": [
            {
                "Name": "Gunther Paul",
                "Email": "none",
                "Affiliation": []
            },
            {
                "Name": "Nils Darwin Abele",
                "Email": "none",
                "Affiliation": []
            },
            {
                "Name": "Karsten Kluth",
                "Email": "none",
                "Affiliation": []
            }
        ],
        "Affiliations": {},
        "Abstract": "TECHNICAL ABSTRACT Background: Industry 4.0 presents itself as an ecosystem; a collection of elements endowed with Cyber-Physical Systems and Augmented Reality/Virtual Reality devices, which are connected through the Internet of Things, and uploaded to Cloud Platforms for analysis, knowledge extr

# REVIEW ABSTRACT

In [56]:
for article in articles:
    print("*************************************************************************")
    print(article['Tittle'])
    print(article['Abstract'])
    print("\n\n\n")

*************************************************************************
A Review and Qualitative Meta-Analysis of Digital Human Modeling and Cyber-Physical-Systems in Ergonomics 4.0
TECHNICAL ABSTRACT
Background: Industry 4.0 presents itself as an ecosystem; a collection of elements endowed
with Cyber-Physical Systems and Augmented Reality/Virtual Reality devices, which are
connected through the Internet of Things, and uploaded to Cloud Platforms for analysis,
knowledge extraction, and diagnostics through Cognitive Computing based on a large amount of
data. The concept is centered around data: managing data, analyzing data, and controlling
data. Many factors influence this interconnected working environment, and for that reason
planning and implementing the digital transformation implies many challenges. Industry 4.0
and Ergonomics are being integrated using a variety of tools and approaches, thus
supporting the development of an Ergonomics 4.0 concept.
Purpose: This paper reviews st

In [22]:
def insert_line_breaks(text, words_per_line):
    words = text.split()
    lines = [' '.join(words[i:i + words_per_line]) for i in range(0, len(words), words_per_line)]
    return '\n'.join(lines)

# Set the number of words per line
words_per_line = 12  # Adjust this number as needed

# Open a new text file for writing
with open('output.txt', 'w') as file:
    for article in articles:
        file.write("*************************************************************************\n")
        file.write(article['Tittle'] + '\n')

        # # Insert line breaks into the abstract
        # try:
        #     formatted_abstract = insert_line_breaks(article['Abstract'], words_per_line)
        # except:
        #     formatted_abstract = "none"

        formatted_abstract = article['Abstract']
        
        file.write(formatted_abstract + '\n\n\n')
